In [101]:
#import required packages
#basics
import pandas as pd 
import numpy as np
import os
#stats
from scipy import sparse
import scipy.stats as ss

#viz
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud ,STOPWORDS
from PIL import Image

#nlp
import string
import re    #for regex
import nltk
from nltk.corpus import stopwords

#import spacy
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize


#FeatureEngineering
#from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer, TfidfTransformer
#from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, decomposition, ensemble
#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import train_test_split

import  textblob
import xgboost

from textblob import TextBlob
#from nltk.stem import PorterStemmer

In [102]:
%pwd

'C:\\Users\\Akshay\\Desktop\\NLP- Woman Clothing'

In [103]:
os.chdir('C:\\Users\\Akshay\\Desktop\\NLP- Woman Clothing')

In [104]:
cloths = pd.read_csv('Womens Clothing Reviews Data.csv')

In [105]:
cloths.head()

,Product ID,Category,Subcategory1,SubCategory2,Location,Channel,Customer Age,Review Title,Review Text,Rating,Recommend Flag
0,767,Initmates,Intimate,Intimates,Mumbai,Mobile,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1
1,1080,General,Dresses,Dresses,Bangalore,Mobile,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1
2,1077,General,Dresses,Dresses,Gurgaon,Mobile,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0
3,1049,General Petite,Bottoms,Pants,Chennai,Web,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1
4,847,General,Tops,Blouses,Bangalore,Web,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1


In [106]:
cloths = cloths.drop(columns = ['Product ID', 'Review Title'], axis=1)

In [107]:
cloths.rename(columns={'Customer Age':'Customer_Age', 'Review Text':'Review_Text', 'Recommend Flag':'Recommend_Flag' }, inplace=True)

In [108]:
cloths.head()

,Category,Subcategory1,SubCategory2,Location,Channel,Customer_Age,Review_Text,Rating,Recommend_Flag
0,Initmates,Intimate,Intimates,Mumbai,Mobile,33,Absolutely wonderful - silky and sexy and comf...,4,1
1,General,Dresses,Dresses,Bangalore,Mobile,34,Love this dress! it's sooo pretty. i happene...,5,1
2,General,Dresses,Dresses,Gurgaon,Mobile,60,I had such high hopes for this dress and reall...,3,0
3,General Petite,Bottoms,Pants,Chennai,Web,50,"I love, love, love this jumpsuit. it's fun, fl...",5,1
4,General,Tops,Blouses,Bangalore,Web,47,This shirt is very flattering to all due to th...,5,1


In [109]:
cloths.sort_values('Rating')

,Category,Subcategory1,SubCategory2,Location,Channel,Customer_Age,Review_Text,Rating,Recommend_Flag
20289,General Petite,Bottoms,Jeans,Mumbai,Web,37,I purchased these in january and today i am go...,1,0
14201,General,Jackets,Outerwear,Bangalore,Web,26,"This coat is awful, just awful. i bought it ba...",1,0
22346,General,Bottoms,Shorts,Bangalore,Web,30,I followed the other reviews and sized down. n...,1,0
23019,General,Bottoms,Skirts,Mumbai,Web,30,This skirt is definitely not what i was expect...,1,0
18740,General,Dresses,Dresses,Chennai,Mobile,37,"Quality was not what you expect from retailer,...",1,0
...,...,...,...,...,...,...,...,...,...
10078,General Petite,Intimate,Lounge,Bangalore,Mobile,44,"This is the perfect tank. it's super soft, gre...",5,1
10079,General Petite,Intimate,Lounge,Chennai,Web,36,These tanks are very soft. i have two black an...,5,1
10080,General,Tops,Knits,Gurgaon,Mobile,66,Love the cut of this t-shirt. just loose enoug...,5,1
10082,General,Tops,Blouses,Mumbai,Web,39,"But when i saw this top in person, urged by my...",5,1


In [149]:
print(cloths.Rating.value_counts(),"\n In Percentage \n",cloths.Rating.value_counts()/sum(cloths.Rating.value_counts())*100)

5    12527
4     4908
3     2823
2     1549
1      821
Name: Rating, dtype: int64 
 In Percentage 
 5    55.360615
4    21.689942
3    12.475694
2     6.845501
1     3.628248
Name: Rating, dtype: float64


In [111]:
cloths.isna().sum()

Category           14
Subcategory1       14
SubCategory2       14
Location            0
Channel             0
Customer_Age        0
Review_Text       845
Rating              0
Recommend_Flag      0
dtype: int64

In [112]:
cloths.dropna(subset=['Review_Text','Category'],inplace=True)

In [113]:
cloths.isna().sum()

Category          0
Subcategory1      0
SubCategory2      0
Location          0
Channel           0
Customer_Age      0
Review_Text       0
Rating            0
Recommend_Flag    0
dtype: int64

In [117]:
cloths.Review_Text[0]

'Absolutely wonderful - silky and sexy and comfortable'

In [123]:
TextBlob(cloths.Review_Text[0]).sentiment.polarity

0.6333333333333333

In [139]:
cloths.Review_Text[1947]

'This is a beautiful sweater! it is not too heavy and the variation in knit from front to back makes not the same old boring sweater. the color is gorgeous.'

In [140]:
TextBlob(cloths.Review_Text[1947]).sentiment.polarity

0.08571428571428572

In [ ]:
# Check the sentiment polarity score for negative review.

In [134]:
rev1 = 'The clothing quality is bad. Unsatisfied product'

In [135]:
TextBlob(rev1).sentiment.polarity

-0.6999999999999998

In [ ]:
# Check the sentiment polarity score for Review_Text.

In [143]:
cloths['Sentiment_Score'] = cloths["Review_Text"].apply(lambda x: TextBlob(x).sentiment.polarity)
cloths.head()

,Category,Subcategory1,SubCategory2,Location,Channel,Customer_Age,Review_Text,Rating,Recommend_Flag,Sentiment_Score
0,Initmates,Intimate,Intimates,Mumbai,Mobile,33,Absolutely wonderful - silky and sexy and comf...,4,1,0.633333
1,General,Dresses,Dresses,Bangalore,Mobile,34,Love this dress! it's sooo pretty. i happene...,5,1,0.339583
2,General,Dresses,Dresses,Gurgaon,Mobile,60,I had such high hopes for this dress and reall...,3,0,0.073675
3,General Petite,Bottoms,Pants,Chennai,Web,50,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0.550000
4,General,Tops,Blouses,Bangalore,Web,47,This shirt is very flattering to all due to th...,5,1,0.512891
